In [1]:
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd


# Load your data
df_radar = pd.read_csv("CleanedData/player_radar.csv")
df_stats = pd.read_csv("CleanedData/player_stats_cleaned.csv")

# Merge the datasets on the 'player' column without adding suffixes
merged_df = pd.merge(df_stats, df_radar, on='player', how='outer', suffixes=('', '_drop'))

# Drop columns with duplicate names from the second dataframe
df = merged_df.drop(merged_df.filter(regex='_drop$').columns, axis=1)


df.drop(columns=['shots_percentage','passes_percentage'], inplace=True)



In [12]:
# Initialize the Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    dbc.Row([
        dbc.Col(html.Div([
            dcc.Dropdown(
                id='team-dropdown',
                options=[{'label': team, 'value': team} for team in df['team'].dropna().unique()],
                multi=True,
                placeholder="Filter by Team",
            )
        ], style={'margin': '10px'}), width=4),
        
        dbc.Col(html.Div([
            dcc.Dropdown(
                id='position-dropdown',
                options=[{'label': position, 'value': position} for position in df['position'].dropna().unique()],
                multi=False,
                placeholder="Filter by Position",
                searchable=False
            )
        ], style={'margin': '10px'}), width=4),

        dbc.Col(html.Div([
            dcc.Dropdown(
                id='player-dropdown',
                options=[{'label': player, 'value': player} for player in df['player'].dropna().unique()],
                multi=True,
                placeholder="Select Players",
            )
        ], style={'margin': '10px'}), width=8),
        
        dbc.Col(html.Div([
            dcc.Dropdown(
                id='metric_x-dropdown',
                options=df.columns[4:], value='shots',
                multi=False,
                placeholder="Select a metric for x-axis",
                clearable=False,
            )
        ], style={'margin': '10px'}), width=4),
        
        dbc.Col(html.Div([
            dcc.Dropdown(
                id='metric_y-dropdown',
                options=df.columns[4:], value='xg',
                multi=False,
                placeholder="Select a metric for y-axis",
                clearable=False,
            )
        ], style={'margin': '10px'}), width=4),
    ]),
    
    dbc.Row([
        dbc.Col(dcc.Graph(id='scatter-plot'), width=12)
    ]),
])

@app.callback(
  [Output('metric_x-dropdown', 'options'),
   Output('metric_y-dropdown', 'options')],
  [Input('position-dropdown', 'value')]
)
def update_metric_dropdowns(selected_position):
    # Get all unique metrics available for the selected position
    if selected_position:
        available_metrics = df[df['position'] == selected_position].columns[4:].tolist()
        options = [{'label': metric, 'value': metric} for metric in available_metrics]
    else:
        options = df.columns[4:]  # Empty options if no position selected
    return options, options  # Return options for both dropdowns

@app.callback(
  Output('player-dropdown', 'options'),
  [Input('team-dropdown', 'value'),
   Input('position-dropdown', 'value')]
)
def update_player_dropdown(selected_teams, selected_position):
    # Filter data based on selections
    filtered_df = df.copy()  # Make a copy to avoid modifying original df
    if selected_teams:
        filtered_df = filtered_df[filtered_df['team'].isin(selected_teams)]
    if selected_position:
        filtered_df = filtered_df[filtered_df['position'] == selected_position]
  
    # Generate player dropdown options
    player_options = [{'label': player, 'value': player} for player in filtered_df['player'].dropna().unique()]
    return player_options

@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('team-dropdown', 'value'),
     Input('position-dropdown', 'value'),
     Input('player-dropdown', 'value'),
     Input('metric_x-dropdown', 'value'),
     Input('metric_y-dropdown', 'value')]
)
def update_scatter_plot(selected_teams, selected_positions, selected_players, selected_metric_x="xg", selected_metric_y='shots'):
    # Filter data based on selections (handle no selection cases)
    filtered_df = df.copy()
    if selected_teams:
        filtered_df = filtered_df[filtered_df['team'].isin(selected_teams)]
    if selected_positions:
        filtered_df = filtered_df[filtered_df['position'] == selected_positions]
    
    # Assign colors
    color_map = {'Other Players': 'lightgrey'}
    colors = px.colors.qualitative.Set1  # Use Set1 qualitative color scheme from Plotly
    if selected_players:
        for i, player in enumerate(selected_players):
            color_map[player] = colors[i % len(colors)]
    else:
        selected_players = ['Other Players']
    
    filtered_df['color'] = filtered_df['player'].map(color_map).fillna('lightgrey')
    filtered_df['legend_group'] = filtered_df['player'].apply(lambda x: x if x in selected_players else 'Other Players')
    
    # Generate the scatter plot
    fig = px.scatter(
        filtered_df,
        x=selected_metric_x,
        y=selected_metric_y,
        color='legend_group',
        color_discrete_map=color_map,
        hover_name='player',  # Show player name on hover
        hover_data={
            'team': True,
            'position': True,
            selected_metric_x: True,
            selected_metric_y: True,
        },
        title=f'{selected_metric_x} vs. {selected_metric_y} for Selected Players and Teams'
    )

    fig.update_traces(marker={'size': 10})
    fig.update_layout(transition_duration=500)
    return fig

@app.callback(
    Output('player-dropdown', 'value'),
    [Input('scatter-plot', 'clickData')],
    [Input('player-dropdown', 'value')]
)
def update_player_dropdown_on_click(clickData, selected_players):
    if clickData is not None:
        clicked_player = clickData['points'][0]['hovertext']
        if selected_players is None:
            selected_players = []
        if clicked_player not in selected_players:
            selected_players.append(clicked_player)
    return selected_players

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8055)

In [13]:
import threading
from IPython.display import display, HTML

# Function to run the Dash app
def run_dash():
    app.run_server(debug=False, use_reloader=False)

# Start the Dash app in a separate thread
threading.Thread(target=run_dash).start()

# Display the link to open the app in the browser
display(HTML(f"""
    <a href="http://127.0.0.1:8055/" target="_blank">
        Open Dash App
    </a>
"""))

Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.
